
This example shows how to extract json data from invoices using the Form Recognizer SDK.  

 
For more information about how to use the Form Recognizer SDK, see:  
https://docs.microsoft.com/en-us/azure/cognitive-services/form-recognizer/quickstarts/client-library?tabs=preview%2Cv2-1&pivots=programming-language-python  



In [12]:
import os
from dotenv import load_dotenv
import os
import json
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, AnalyzeDocumentRequest

load_dotenv()
endpoint = os.environ["AZURE_FORM_RECOGNIZER_ENDPOINT"]
key = os.environ["AZURE_FORM_RECOGNIZER_KEY"]

In [13]:
def extract_fields(json_object, property_name, property_value):
    if property_value:
        json_object[property_name] = {}
        json_object[property_name]["value"] = property_value.value
        json_object[property_name]["confidence"] =property_value.confidence

def extract_address(json_object, property_name, property_value):
    if property_value:
        json_object[property_name] = {}
        json_object[property_name]["value"] = {}
        json_object[property_name]["value"]["city"] = property_value.value.city
        json_object[property_name]["value"]["country_region"] = property_value.value.country_region
        json_object[property_name]["value"]["house_number"] = property_value.value.house_number
        json_object[property_name]["value"]["po_box"] = property_value.value.po_box
        json_object[property_name]["value"]["postal_code"] = property_value.value.postal_code
        json_object[property_name]["value"]["road"] = property_value.value.road
        json_object[property_name]["value"]["state"] = property_value.value.state
        json_object[property_name]["value"]["street_address"] = property_value.value.street_address
        json_object[property_name]["confidence"] =property_value.confidence

def extract_date(json_object, property_name, property_value):
    if property_value:
        json_object[property_name] = {}
        json_object[property_name]["value"] = {}
        json_object[property_name]["value"]["day"] = property_value.value.day
        json_object[property_name]["value"]["month"] = property_value.value.month
        json_object[property_name]["value"]["year"] = property_value.value.year
        json_object[property_name]["confidence"] =property_value.confidence


def extract_currency(json_object, property_name, property_value):
    if property_value:
        json_object[property_name] = {}
        json_object[property_name]["value"] = {}
        json_object[property_name]["value"]["amount"] = property_value.value.amount
        json_object[property_name]["value"]["symbol"] = property_value.value.symbol
        json_object[property_name]["confidence"] =property_value.confidence



def analyze_invoices():
    document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )

    for filename in os.listdir("./invoices/"):
        idx = 0
        with open(os.path.join("./invoices/", filename), 'rb') as f:
            poller = document_analysis_client.begin_analyze_document(
                "prebuilt-invoice", document=f, locale="en-US"
        )   

        invoice = poller.result().documents[0]
        print("Invoice #{}".format(idx+1))
        invoice_json = {}
        invoice_json["file_name"] = filename

        extract_fields(invoice_json, "VendorName", invoice.fields.get("VendorName"))
        extract_address(invoice_json, "VendorAddress", invoice.fields.get("VendorAddress"))
        extract_fields(invoice_json, "VendorAddressRecipient", invoice.fields.get("VendorAddressRecipient"))
        extract_fields(invoice_json, "CustomerName", invoice.fields.get("CustomerName"))
        extract_fields(invoice_json, "CustomerId", invoice.fields.get("CustomerId"))
        extract_address(invoice_json, "CustomerAddress", invoice.fields.get("CustomerAddress"))
        extract_fields(invoice_json, "CustomerAddressRecipient", invoice.fields.get("CustomerAddressRecipient"))
        extract_fields(invoice_json, "InvoiceId", invoice.fields.get("InvoiceId"))
        extract_date(invoice_json, "InvoiceDate", invoice.fields.get("InvoiceDate"))
        extract_currency(invoice_json, "InvoiceTotal", invoice.fields.get("InvoiceTotal"))
        extract_date(invoice_json, "DueDate", invoice.fields.get("DueDate"))
        extract_fields(invoice_json, "PurchaseOrder", invoice.fields.get("PurchaseOrder"))
        extract_address(invoice_json, "BillingAddress", invoice.fields.get("BillingAddress"))
        extract_fields(invoice_json, "BillingAddressRecipient", invoice.fields.get("BillingAddressRecipient"))
        extract_address(invoice_json, "ShippingAddress", invoice.fields.get("ShippingAddress"))
        extract_fields(invoice_json, "ShippingAddressRecipient", invoice.fields.get("ShippingAddressRecipient"))
        invoice_json["Items"] = []  

        if (invoice.fields.get("Items")):
            for idx, item in enumerate(invoice.fields.get("Items").value):
                item_json = {}
                item = invoice.fields.get("Items").value[idx]
                extract_fields(item_json, "Name", item.value.get("Name"))
                extract_fields(item_json, "Description", item.value.get("Description"))
                extract_fields(item_json, "Quantity", item.value.get("Quantity"))
                extract_fields(item_json, "Unit", item.value.get("Unit"))
                extract_currency(item_json, "UnitPrice", item.value.get("UnitPrice"))
                extract_fields(item_json, "ProductCode", item.value.get("ProductCode"))
                extract_date(item_json, "Date", item.value.get("Date"))
                extract_currency(item_json, "Tax", item.value.get("Tax"))
                extract_currency(item_json, "Amount", item.value.get("Amount"))
                invoice_json["Items"].append(item_json)

        extract_currency(invoice_json, "SubTotal", invoice.fields.get("SubTotal"))
        extract_currency(invoice_json, "TotalTax", invoice.fields.get("TotalTax"))
        extract_currency(invoice_json, "PreviousUnpaidBalance", invoice.fields.get("PreviousUnpaidBalance"))
        extract_currency(invoice_json, "AmountDue", invoice.fields.get("AmountDue"))
        extract_date(invoice_json, "ServiceStartDate", invoice.fields.get("ServiceStartDate"))
        extract_date(invoice_json, "ServiceEndDate", invoice.fields.get("ServiceEndDate"))
        extract_date(invoice_json, "ServiceDate", invoice.fields.get("ServiceDate"))
        extract_address(invoice_json, "ServiceAddress", invoice.fields.get("ServiceAddress"))
        extract_fields(invoice_json, "ServiceAddressRecipient", invoice.fields.get("ServiceAddressRecipient"))
        extract_address(invoice_json, "RemittanceAddress", invoice.fields.get("RemittanceAddress"))
        extract_fields(invoice_json, "RemittanceAddressRecipient", invoice.fields.get("RemittanceAddressRecipient"))
        print(json.dumps(invoice_json))
        idx += 1

In [ ]:
analyze_invoices()

In [ ]:
# coding: utf-8

# -------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License. See License.txt in the project root for
# license information.
# --------------------------------------------------------------------------

"""
FILE: sample_analyze_invoices.py

DESCRIPTION:
    This sample demonstrates how to analyze invoices.

    See fields found on a invoice here:
    https://aka.ms/invoice-field-schema

PREREQUISITES:
    The following prerequisites are necessary to run the code. For more details, please visit the "How-to guides" link: https://aka.ms/how-to-guide

    -------Python and IDE------
    1) Install Python 3.7 or later (https://www.python.org/), which should include pip (https://pip.pypa.io/en/stable/).
    2) Install the latest version of Visual Studio Code (https://code.visualstudio.com/) or your preferred IDE. 
    
    ------Azure AI services or Document Intelligence resource------ 
    Create a single-service (https://aka.ms/single-service) or multi-service (https://aka.ms/multi-service) resource.
    You can use the free pricing tier (F0) to try the service and upgrade to a paid tier for production later.
    
    ------Get the key and endpoint------
    1) After your resource is deployed, select "Go to resource". 
    2) In the left navigation menu, select "Keys and Endpoint". 
    3) Copy one of the keys and the Endpoint for use in this sample. 
    
    ------Set your environment variables------
    At a command prompt, run the following commands, replacing <yourKey> and <yourEndpoint> with the values from your resource in the Azure portal.
    1) For Windows:
       setx DOCUMENTINTELLIGENCE_API_KEY <yourKey>
       setx DOCUMENTINTELLIGENCE_ENDPOINT <yourEndpoint>
       • You need to restart any running programs that read the environment variable.
    2) For macOS:
       export key=<yourKey>
       export endpoint=<yourEndpoint>
       • This is a temporary environment variable setting method that only lasts until you close the terminal session. 
       • To set an environment variable permanently, visit: https://aka.ms/set-environment-variables-for-macOS
    3) For Linux:
       export DOCUMENTINTELLIGENCE_API_KEY=<yourKey>
       export DOCUMENTINTELLIGENCE_ENDPOINT=<yourEndpoint>
       • This is a temporary environment variable setting method that only lasts until you close the terminal session. 
       • To set an environment variable permanently, visit: https://aka.ms/set-environment-variables-for-Linux
       
    ------Set up your programming environment------
    At a command prompt,run the following code to install the Azure AI Document Intelligence client library for Python with pip:
    pip install azure-ai-documentintelligence --pre
    
    ------Create your Python application------
    1) Create a new Python file called "sample_analyze_invoices.py" in an editor or IDE.
    2) Open the "sample_analyze_invoices.py" file and insert the provided code sample into your application.
    3) At a command prompt, use the following code to run the Python code: 
       python sample_analyze_invoices.py
"""

import os


def analyze_invoice():
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.documentintelligence import DocumentIntelligenceClient
    from azure.ai.documentintelligence.models import AnalyzeResult, AnalyzeDocumentRequest

    # For how to obtain the endpoint and key, please see PREREQUISITES above.
    endpoint = os.environ["DOCUMENTINTELLIGENCE_ENDPOINT"]
    key = os.environ["DOCUMENTINTELLIGENCE_API_KEY"]

    document_intelligence_client = DocumentIntelligenceClient(endpoint=endpoint, credential=AzureKeyCredential(key))

    # Analyze a document at a URL:
    invoiceUrl = "https://github.com/Azure-Samples/cognitive-services-REST-api-samples/raw/master/curl/form-recognizer/rest-api/invoice.pdf"
    # Replace with your actual invoiceUrl:
    # If you use the URL of a public website, to find more URLs, please visit: https://aka.ms/more-URLs 
    # If you analyze a document in Blob Storage, you need to generate Public SAS URL, please visit: https://aka.ms/create-sas-tokens
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-invoice",
        AnalyzeDocumentRequest(url_source=invoiceUrl)
    )       
    
    # # If analyzing a local document, remove the comment markers (#) at the beginning of these 8 lines.
    # # Delete or comment out the part of "Analyze a document at a URL" above.
    # # Replace <path to your sample file>  with your actual file path.
    # path_to_sample_document = "<path to your sample file>"
    # with open(path_to_sample_document, "rb") as f:
    #     poller = document_intelligence_client.begin_analyze_document(
    #         "prebuilt-invoice", analyze_request=f, locale="en-US", content_type="application/octet-stream"
    #     )
    invoices: AnalyzeResult = poller.result()   

    # [START analyze_invoices]
    if invoices.documents:
        for idx, invoice in enumerate(invoices.documents):
            print(f"--------Analyzing invoice #{idx + 1}--------")
            if invoice.fields:
                vendor_name = invoice.fields.get("VendorName")
                if vendor_name:
                    print(f"Vendor Name: {vendor_name.get('content')} has confidence: {vendor_name.get('confidence')}")
                vendor_address = invoice.fields.get("VendorAddress")
                if vendor_address:
                    print(
                        f"Vendor Address: {vendor_address.get('content')} has confidence: {vendor_address.get('confidence')}"
                    )
                vendor_address_recipient = invoice.fields.get("VendorAddressRecipient")
                if vendor_address_recipient:
                    print(
                        f"Vendor Address Recipient: {vendor_address_recipient.get('content')} has confidence: {vendor_address_recipient.get('confidence')}"
                    )
                customer_name = invoice.fields.get("CustomerName")
                if customer_name:
                    print(
                        f"Customer Name: {customer_name.get('content')} has confidence: {customer_name.get('confidence')}"
                    )
                customer_id = invoice.fields.get("CustomerId")
                if customer_id:
                    print(f"Customer Id: {customer_id.get('content')} has confidence: {customer_id.get('confidence')}")
                customer_address = invoice.fields.get("CustomerAddress")
                if customer_address:
                    print(
                        f"Customer Address: {customer_address.get('content')} has confidence: {customer_address.get('confidence')}"
                    )
                customer_address_recipient = invoice.fields.get("CustomerAddressRecipient")
                if customer_address_recipient:
                    print(
                        f"Customer Address Recipient: {customer_address_recipient.get('content')} has confidence: {customer_address_recipient.get('confidence')}"
                    )
                invoice_id = invoice.fields.get("InvoiceId")
                if invoice_id:
                    print(f"Invoice Id: {invoice_id.get('content')} has confidence: {invoice_id.get('confidence')}")
                invoice_date = invoice.fields.get("InvoiceDate")
                if invoice_date:
                    print(
                        f"Invoice Date: {invoice_date.get('content')} has confidence: {invoice_date.get('confidence')}"
                    )
                invoice_total = invoice.fields.get("InvoiceTotal")
                if invoice_total:
                    print(
                        f"Invoice Total: {invoice_total.get('content')} has confidence: {invoice_total.get('confidence')}"
                    )
                due_date = invoice.fields.get("DueDate")
                if due_date:
                    print(f"Due Date: {due_date.get('content')} has confidence: {due_date.get('confidence')}")
                purchase_order = invoice.fields.get("PurchaseOrder")
                if purchase_order:
                    print(
                        f"Purchase Order: {purchase_order.get('content')} has confidence: {purchase_order.get('confidence')}"
                    )
                billing_address = invoice.fields.get("BillingAddress")
                if billing_address:
                    print(
                        f"Billing Address: {billing_address.get('content')} has confidence: {billing_address.get('confidence')}"
                    )
                billing_address_recipient = invoice.fields.get("BillingAddressRecipient")
                if billing_address_recipient:
                    print(
                        f"Billing Address Recipient: {billing_address_recipient.get('content')} has confidence: {billing_address_recipient.get('confidence')}"
                    )
                shipping_address = invoice.fields.get("ShippingAddress")
                if shipping_address:
                    print(
                        f"Shipping Address: {shipping_address.get('content')} has confidence: {shipping_address.get('confidence')}"
                    )
                shipping_address_recipient = invoice.fields.get("ShippingAddressRecipient")
                if shipping_address_recipient:
                    print(
                        f"Shipping Address Recipient: {shipping_address_recipient.get('content')} has confidence: {shipping_address_recipient.get('confidence')}"
                    )
                print("Invoice items:")
                items = invoice.fields.get("Items")
                if items:
                    for idx, item in enumerate(items.get("valueArray")):
                        print(f"...Item #{idx + 1}")
                        item_description = item.get("valueObject").get("Description")
                        if item_description:
                            print(
                                f"......Description: {item_description.get('content')} has confidence: {item_description.get('confidence')}"
                            )
                        item_quantity = item.get("valueObject").get("Quantity")
                        if item_quantity:
                            print(
                                f"......Quantity: {item_quantity.get('content')} has confidence: {item_quantity.get('confidence')}"
                            )
                        unit = item.get("valueObject").get("Unit")
                        if unit:
                            print(f"......Unit: {unit.get('content')} has confidence: {unit.get('confidence')}")
                        unit_price = item.get("valueObject").get("UnitPrice")
                        if unit_price:
                            unit_price_code = (
                                unit_price.get("valueCurrency").get("currencyCode")
                                if unit_price.get("valueCurrency").get("currencyCode")
                                else ""
                            )
                            print(
                                f"......Unit Price: {unit_price.get('content')}{unit_price_code} has confidence: {unit_price.get('confidence')}"
                            )
                        product_code = item.get("valueObject").get("ProductCode")
                        if product_code:
                            print(
                                f"......Product Code: {product_code.get('content')} has confidence: {product_code.get('confidence')}"
                            )
                        item_date = item.get("valueObject").get("Date")
                        if item_date:
                            print(
                                f"......Date: {item_date.get('content')} has confidence: {item_date.get('confidence')}"
                            )
                        tax = item.get("valueObject").get("Tax")
                        if tax:
                            print(f"......Tax: {tax.get('content')} has confidence: {tax.get('confidence')}")
                        amount = item.get("valueObject").get("Amount")
                        if amount:
                            print(f"......Amount: {amount.get('content')} has confidence: {amount.get('confidence')}")
                subtotal = invoice.fields.get("SubTotal")
                if subtotal:
                    print(f"Subtotal: {subtotal.get('content')} has confidence: {subtotal.get('confidence')}")
                total_tax = invoice.fields.get("TotalTax")
                if total_tax:
                    print(f"Total Tax: {total_tax.get('content')} has confidence: {total_tax.get('confidence')}")
                previous_unpaid_balance = invoice.fields.get("PreviousUnpaidBalance")
                if previous_unpaid_balance:
                    print(
                        f"Previous Unpaid Balance: {previous_unpaid_balance.get('content')} has confidence: {previous_unpaid_balance.get('confidence')}"
                    )
                amount_due = invoice.fields.get("AmountDue")
                if amount_due:
                    print(f"Amount Due: {amount_due.get('content')} has confidence: {amount_due.get('confidence')}")
                service_start_date = invoice.fields.get("ServiceStartDate")
                if service_start_date:
                    print(
                        f"Service Start Date: {service_start_date.get('content')} has confidence: {service_start_date.get('confidence')}"
                    )
                service_end_date = invoice.fields.get("ServiceEndDate")
                if service_end_date:
                    print(
                        f"Service End Date: {service_end_date.get('content')} has confidence: {service_end_date.get('confidence')}"
                    )
                service_address = invoice.fields.get("ServiceAddress")
                if service_address:
                    print(
                        f"Service Address: {service_address.get('content')} has confidence: {service_address.get('confidence')}"
                    )
                service_address_recipient = invoice.fields.get("ServiceAddressRecipient")
                if service_address_recipient:
                    print(
                        f"Service Address Recipient: {service_address_recipient.get('content')} has confidence: {service_address_recipient.get('confidence')}"
                    )
                remittance_address = invoice.fields.get("RemittanceAddress")
                if remittance_address:
                    print(
                        f"Remittance Address: {remittance_address.get('content')} has confidence: {remittance_address.get('confidence')}"
                    )
                remittance_address_recipient = invoice.fields.get("RemittanceAddressRecipient")
                if remittance_address_recipient:
                    print(
                        f"Remittance Address Recipient: {remittance_address_recipient.get('content')} has confidence: {remittance_address_recipient.get('confidence')}"
                    )
    # [END analyze_invoices]


if __name__ == "__main__":
    from azure.core.exceptions import HttpResponseError
    from dotenv import find_dotenv, load_dotenv

    try:
        load_dotenv(find_dotenv())
        analyze_invoice()
    except HttpResponseError as error:
        # Examples of how to check an HttpResponseError
        # Check by error code:
        if error.error is not None:
            if error.error.code == "InvalidImage":
                print(f"Received an invalid image error: {error.error}")
            if error.error.code == "InvalidRequest":
                print(f"Received an invalid request error: {error.error}")
            # Raise the error again after printing it
            raise
        # If the inner error is None and then it is possible to check the message to get more information:
        if "Invalid request".casefold() in error.message.casefold():
            print(f"Uh-oh! Seems there was an invalid request: {error}")
        # Raise the error again
        raise

# Next steps:
# Learn more about Invoice model: https://aka.ms/concept-invoice
# Find more sample code: https://aka.ms/doc-intelligence-samples